# Word Embeddings : le modèle Word2Vec

## Imports

In [1]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

ModuleNotFoundError: No module named 'gensim'

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [2]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [3]:
infile = f"../data/sents.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [4]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [5]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [6]:
len(bigram_phrases.vocab.keys())

15852820

Prenons une clé au hasard :

In [7]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

1q


Le dictionnaire indique le score de cette coocurrence :

In [8]:
bigram_phrases.vocab[key_]

488

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [9]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [10]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [11]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [ ]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [ ]:
print(corpus[:100])

[['mi', 'imnri', 'r', 'i', '<<', 'i', 'i', 'hmu', "'", 'i', '/', 'tx', "-'", 'l', ':', 'marche', 'tenu', 'hors', 'villa', ',', 'la', '9', '.'], ['--', 'u', 'a', 'ete', 'vaain', 'si', 'teicj', '>>', 'm', 'races', 'indigenes', 'de', 'fr', '.'], ['31', '<)', 'a', '5s', "'", 'k', '131', 'de', '.'], ['rasa', 'iichakdui', "'", 'te', ',', 'do', '(', 'r', '.', '3s0', 'h', '710', '.', 'taureaux', 'iallsenas', ',>', 'ia', 'u', '\\', '--', 'a', '--', ';', '0ii', '.'], ['hollandais', ',', 'dufr', '.'], ['0', '.'], ['--', 'a', '9', '.--', 'la', 'idto', '-', 'vachei', 'laitieres', ':', 'bn', 'vante', '1q', '.'], ['vendues', '3', '\\', 'au', 'prix', 'la', '410', 'a', '*', '<<', 'i', 'h', '\\;', 'genisses', ',', 'kl', '.'], ["'.", '9', '.'], ['i', 'l', '.', '2', 'i', '.', 'id', '.'], ['da', '370', 'i', '6lutr', '.'], ['marche', 'a', '<', 'u', 'porcs', '.'], ['--', 'categorie', 'de', 'lt', 'ilashtya', ':', "'", '237', 'on', 'vente', ';', 'vendus', '1', 'm', '.', 'do', "'", '2', 'i', '.--', 'a', ';:,', 

## Entrainement d'un modèle Word2Vec sur ce corpus

In [ ]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et 5 après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus (on peut le mettre à plus avec un très gros corpus)
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: total: 1min 17s
Wall time: 29.8 s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [18]:
outfile = f"../data/newspapers.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [19]:
model = Word2Vec.load("../data/newspapers.model")

### Imprimer le vecteur d'un terme

In [21]:
model.wv["ministre"]

array([-2.6399488 ,  1.5406489 , -0.8392589 ,  2.4930084 ,  1.6386236 ,
        0.15226486, -1.6433192 ,  0.8803645 , -1.9486518 , -2.6073043 ,
        2.7060695 ,  4.5620933 ,  0.10546511, -1.869356  , -4.8620796 ,
       -1.3932638 , -0.97642106, -0.9568132 , -1.9307603 ,  0.8648233 ,
        3.1736832 , -0.22823694,  1.2658305 , -1.2499422 , -0.8875545 ,
        0.12192031, -0.43442303,  0.26246613, -2.2754087 , -1.4227809 ,
       -0.7125336 ,  0.35338837], dtype=float32)

### Calculer la similarité entre deux termes

In [22]:
model.wv.similarity("ministre", "roi")

0.56487834

### Chercher les mots les plus proches d'un terme donné

In [23]:
model.wv.most_similar("ministre", topn=10)

[('secretaire', 0.9073051810264587),
 ('president', 0.8692206144332886),
 ('membre', 0.8604556322097778),
 ('en_remplacement', 0.8563957214355469),
 ('vice_-_president', 0.8453834652900696),
 ('directeur', 0.8450940847396851),
 ('professeur', 0.8420858979225159),
 ('adjoint', 0.8359804749488831),
 ('bourgmestre', 0.8348686695098877),
 ('conseiller', 0.8315735459327698)]

### Faire des recherches complexes à travers l'espace vectoriel

In [24]:
print(model.wv.most_similar(positive=['paris', 'londres'], negative=['belgique']))

[('la_monnaie', 0.8212859630584717), ('au_theatre', 0.8006280064582825), ('ce_matin', 0.7882864475250244), ('berlin', 0.7768507599830627), ('marseille', 0.7672545313835144), ('dimanche', 0.7663719654083252), ('ce_soir', 0.7549272775650024), ('hier_soir', 0.7544046640396118), ('new_-_york', 0.7509400248527527), ('minuit', 0.7490603923797607)]
